In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Preprocessing



In [49]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

median_masked = pd.read_csv('/content/drive/MyDrive/ML Semester Project/TimeSeries_median_masked.csv')
maximum = pd.read_csv('/content/drive/MyDrive/ML Semester Project/TimeSeries_maximum.csv')
minimum = pd.read_csv('/content/drive/MyDrive/ML Semester Project/TimeSeries_minimum.csv')
average= pd.read_csv('/content/drive/MyDrive/ML Semester Project/TimeSeries_average_masked.csv')
population = pd.read_csv('/content/drive/MyDrive/ML Semester Project/GDP_Population_Energy/GDP_Population_Energy/population_updated.csv')

# Clean column names
for df in [median_masked, maximum, minimum, average, population]:
    df.columns = df.columns.str.strip()

# Rename population column to match others
population.rename(columns={'Country Name': 'Country'}, inplace=True)

# List of countries
countries = median_masked['Country'].unique()

# Initialize a list to store forecasted results
forecast_results_arima = []
forecast_results_sarima = []
forecast_results_hw = []

# Define years to use
years_all = [str(y) for y in range(2013, 202)]      # 2013–2023
years_train = [str(y) for y in range(2013, 2023)]    # 2013–2022
forecast_year = '2023'


##2. Model Training and Forecasting

ARIMA Model:

In [57]:
for country_name in countries:
    try:
        # Filter country-specific rows (same as your current code)
        masked_row = median_masked[median_masked['Country'] == country_name]
        max_row = maximum[maximum['Country'] == country_name]
        min_row = minimum[minimum['Country'] == country_name]
        med_row = average[average['Country'] == country_name]
        pop_row = population[population['Country'] == country_name]

        # Skip if any of them is missing
        if masked_row.empty or max_row.empty or min_row.empty or med_row.empty or pop_row.empty:
            print(f"Skipping {country_name} due to missing data.")
            continue

        # Use the year columns (ensure match)
        band_cols = [str(y) for y in range(2013, 2023)]

        # Extract values for ARIMA (flattened)
        exog = pd.DataFrame({
            'masked': masked_row[band_cols].values.flatten(),
            'max': max_row[band_cols].values.flatten(),
            'min': min_row[band_cols].values.flatten(),
            'median': med_row[band_cols].values.flatten(),
        }, index=pd.Index(years_train, name='Year'))

        y = pd.Series(pop_row[band_cols].values.flatten(), index=years_train)

        # Fit ARIMA model
        model_arima = ARIMA(y, exog=exog, order=(1,1,0))
        model_arima_fit = model_arima.fit()

        # Forecast for 2023 using last known exogenous values (from 2022)
        last_exog = exog.iloc[-1].values.reshape(1, -1)
        future_exog = pd.DataFrame(last_exog, columns=exog.columns, index=[forecast_year])

        forecast_2023_arima = model_arima_fit.forecast(steps=1, exog=future_exog)

        forecast_results_arima.append({
            'Country': country_name,
            'Forecast_2023_ARIMA': forecast_2023_arima[0]
        })

    except Exception as e:
        print(f"ARIMA Model failed for {country_name}: {e}")
        continue


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_ARIMA': forecast_2023_arima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/ts

Skipping United Republic of Tanzania due to missing data.


<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_ARIMA': forecast_2023_arima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__get

Skipping Democratic Republic of the Congo due to missing data.


<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_ARIMA': forecast_2023_arima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__get

...

<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_ARIMA': forecast_2023_arima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__get

Skipping Bosnia & Herzegovina due to missing data.


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_ARIMA': forecast_2023_arima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmod

Skipping Micronesia (Federated States of) due to missing data.


<ipython-input-57-4d0d41ea10a1>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_ARIMA': forecast_2023_arima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.

SARIMA

In [59]:
for country_name in countries:
    try:
        # Filter country-specific rows (same as your current code)
        masked_row = median_masked[median_masked['Country'] == country_name]
        max_row = maximum[maximum['Country'] == country_name]
        min_row = minimum[minimum['Country'] == country_name]
        med_row = average[average['Country'] == country_name]
        pop_row = population[population['Country'] == country_name]

        # Skip if any of them is missing
        if masked_row.empty or max_row.empty or min_row.empty or med_row.empty or pop_row.empty:
            print(f"Skipping {country_name} due to missing data.")
            continue

        # Extract values for SARIMA (same as your current code)
        band_cols = [str(y) for y in range(2013, 2023)]

        # Extract values
        exog = pd.DataFrame({
            'masked': masked_row[band_cols].values.flatten(),
            'max': max_row[band_cols].values.flatten(),
            'min': min_row[band_cols].values.flatten(),
            'median': med_row[band_cols].values.flatten(),
        }, index=pd.Index(years_train, name='Year'))

        y = pd.Series(pop_row[band_cols].values.flatten(), index=years_train)

        # Fit SARIMAX model (same as your current code)
        model_sarima = SARIMAX(y, exog=exog, order=(1, 1, 0))
        model_sarima_fit = model_sarima.fit(disp=False)

        # Forecast for 2023 using last known exogenous values
        last_exog = exog.iloc[-1].values.reshape(1, -1)
        future_exog = pd.DataFrame(last_exog, columns=exog.columns, index=[forecast_year])

        forecast_2023_sarima = model_sarima_fit.forecast(steps=1, exog=future_exog)

        forecast_results_sarima.append({
            'Country': country_name,
            'Forecast_2023_SARIMA': forecast_2023_sarima[0]
        })

    except Exception as e:
        print(f"SARIMA Model failed for {country_name}: {e}")
        continue


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_SARIMA': forecast_2023_sarima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/

Skipping United Republic of Tanzania due to missing data.


<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_SARIMA': forecast_2023_sarima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


Skipping Democratic Republic of the Congo due to missing data.


<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_SARIMA': forecast_2023_sarima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


...

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_SARIMA': forecast_2023_sarima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/

Skipping Bosnia & Herzegovina due to missing data.


<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_SARIMA': forecast_2023_sarima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


Skipping Micronesia (Federated States of) due to missing data.


<ipython-input-59-20cefc5f78cc>:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Forecast_2023_SARIMA': forecast_2023_sarima[0]
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
<ipython-input-59-20cefc5f78cc>:40: FutureWarn

## Model Evaluation for ARIMA and SARIMA

In [60]:
# Merge results for ARIMA, SARIMA, and Holt-Winters with ground truth 2023 population
forecast_df_arima = pd.DataFrame(forecast_results_arima)
forecast_df_sarima = pd.DataFrame(forecast_results_sarima)

# Merge with ground truth population
merged_arima = forecast_df_arima.merge(population[['Country', '2023']], on='Country', how='inner')
merged_sarima = forecast_df_sarima.merge(population[['Country', '2023']], on='Country', how='inner')

# Rename columns for clarity
merged_arima.rename(columns={'2023': 'GroundTruth_2023'}, inplace=True)
merged_sarima.rename(columns={'2023': 'GroundTruth_2023'}, inplace=True)

# Drop missing values
merged_arima.dropna(subset=['Forecast_2023_ARIMA', 'GroundTruth_2023'], inplace=True)
merged_sarima.dropna(subset=['Forecast_2023_SARIMA', 'GroundTruth_2023'], inplace=True)

# Evaluation metrics function
def evaluate_model(merged_data, forecast_col):
    y_true = merged_data['GroundTruth_2023']
    y_pred = merged_data[forecast_col]

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    return r2, mae, rmse

# Evaluate each model
r2_arima, mae_arima, rmse_arima = evaluate_model(merged_arima, 'Forecast_2023_ARIMA')
r2_sarima, mae_sarima, rmse_sarima = evaluate_model(merged_sarima, 'Forecast_2023_SARIMA')

# Print results
print(f"ARIMA: R² = {r2_arima:.4f}, MAE = {mae_arima:.2f}, RMSE = {rmse_arima:.2f}")
print(f"SARIMA: R² = {r2_sarima:.4f}, MAE = {mae_sarima:.2f}, RMSE = {rmse_sarima:.2f}")


ARIMA: R² = 0.9999, MAE = 407218.05, RMSE = 1227851.75
SARIMA: R² = 0.9999, MAE = 354407.22, RMSE = 1169024.06


## Prophet:

In [32]:
pip install prophet

In [35]:
median_df = pd.read_csv('/content/drive/MyDrive/ML Semester Project/TimeSeries_median_masked.csv')
max_df = pd.read_csv('/content/drive/MyDrive/ML Semester Project/TimeSeries_maximum.csv')
min_df = pd.read_csv('/content/drive/MyDrive/ML Semester Project/TimeSeries_minimum.csv')
average_df= pd.read_csv('/content/drive/MyDrive/ML Semester Project/TimeSeries_average_masked.csv')
population = pd.read_csv('/content/drive/MyDrive/ML Semester Project/GDP_Population_Energy/GDP_Population_Energy/population_updated.csv')
# Drop 2024 column if present
median_df = median_df.drop(['2024'], axis=1)
max_df = max_df.drop(['2024'], axis=1)
min_df = min_df.drop(['2024'], axis=1)
average_df = average_df.drop(['2024'], axis=1)

# Rename columns to match format
median_df.columns = ['Country'] + [f'median_{year}' for year in range(2013, 2024)]
max_df.columns = ['Country'] + [f'max_{year}' for year in range(2013, 2024)]
min_df.columns = ['Country'] + [f'min_{year}' for year in range(2013, 2024)]
average_df.columns = ['Country'] + [f'average_{year}' for year in range(2013, 2024)]

# Prepare population dataset
population.rename(columns={'Country Name': 'Country'}, inplace=True)

# Ensure population data has one column for each year (2013 to 2023)
# Assuming 'Country' column is correct and there is one column for each year in population dataset
population_columns = ['Country'] + [str(year) for year in range(2013, 2024)]
population = population[population_columns]

# Merge dataframes on 'Country' using left join to ensure all countries are included
combined_df = pd.merge(median_df, max_df, on='Country', how='left')
combined_df = pd.merge(combined_df, min_df, on='Country', how='left')
combined_df = pd.merge(combined_df, average_df, on='Country', how='left')
combined_df = pd.merge(combined_df, population, on='Country', how='left')

# Reshape data to desired format: First, melt to long format
melted_df = pd.melt(combined_df, id_vars=['Country'], var_name='year_type', value_name='value')

# Extract year and type (median, max, min, average, population) from the 'year_type' column
melted_df['year'] = melted_df['year_type'].str.extract(r'(\d{4})').astype(int)
melted_df['type'] = melted_df['year_type'].str.extract(r'([a-zA-Z]+)')

# Pivot the table to get the desired format
pivot_df = melted_df.pivot_table(index=['Country', 'year'], columns='type', values='value', aggfunc='first')

# Reset index for cleaner table
pivot_df.reset_index(inplace=True)

pivot_df = pivot_df.sort_values(by=['Country', 'year'])


In [36]:
pivot_df

type,Country,year,average,max,median,min
0,Afghanistan,2013,92792.313774,1.091875e+06,88539.246767,190723.040993
1,Afghanistan,2014,82986.993879,1.090830e+06,80624.773952,23251.571958
2,Afghanistan,2015,73424.706667,1.054926e+06,70962.230817,45458.317460
3,Afghanistan,2016,71253.367845,8.178813e+05,68843.099611,-1799.889819
4,Afghanistan,2017,81674.809451,1.560382e+06,78654.057851,660310.840753
...,...,...,...,...,...,...
2118,Zimbabwe,2019,49409.112783,7.378561e+05,46998.620320,86689.230187
2119,Zimbabwe,2020,49187.118091,8.230068e+05,47014.545921,169458.072419
2120,Zimbabwe,2021,58046.526323,9.885022e+05,55878.774081,258232.578915
2121,Zimbabwe,2022,64631.855230,1.157561e+06,60541.246624,93032.815369


In [37]:
population_df = population.melt(id_vars=["Country"], var_name="year", value_name="population")

population_df['year'] = population_df['year'].astype(int)

# Merging population data with the merged dataset based on Country and year
final_merged_df = pd.merge(pivot_df, population_df, on=['Country', 'year'], how='left')

final_merged_df

,Country,year,average,max,median,min,population
0,Afghanistan,2013,92792.313774,1.091875e+06,88539.246767,190723.040993,31622704.0
1,Afghanistan,2014,82986.993879,1.090830e+06,80624.773952,23251.571958,32792523.0
2,Afghanistan,2015,73424.706667,1.054926e+06,70962.230817,45458.317460,33831764.0
3,Afghanistan,2016,71253.367845,8.178813e+05,68843.099611,-1799.889819,34700612.0
4,Afghanistan,2017,81674.809451,1.560382e+06,78654.057851,660310.840753,35688935.0
...,...,...,...,...,...,...,...
2118,Zimbabwe,2019,49409.112783,7.378561e+05,46998.620320,86689.230187,15271368.0
2119,Zimbabwe,2020,49187.118091,8.230068e+05,47014.545921,169458.072419,15526888.0
2120,Zimbabwe,2021,58046.526323,9.885022e+05,55878.774081,258232.578915,15797210.0
2121,Zimbabwe,2022,64631.855230,1.157561e+06,60541.246624,93032.815369,16069056.0


In [38]:
df = final_merged_df.copy()

In [40]:
from prophet import Prophet
df = final_merged_df.copy()
results = []

# Loop through each country
for country in df['Country'].unique():
    df_country = df[df['Country'] == country].copy()

    # Ensure 'ds' column exists
    df_country['ds'] = pd.to_datetime(df_country['year'].astype(str) + '-01-01')

    # Drop 2023 population from training data
    train_df = df_country[df_country['year'] < 2023].copy()
    train_df['y'] = train_df['population']

    # Initialize Prophet and add regressors
    model = Prophet(yearly_seasonality=True)
    for reg in ['average', 'max', 'median', 'min']:
        model.add_regressor(reg)

    try:
        # Fit model with regressors
        model.fit(train_df[['ds', 'y', 'average', 'max', 'median', 'min']])
    except Exception as e:
        print(f"Skipping {country} due to model error: {e}")
        continue

    # Prepare the 2023 row for prediction
    df_2023 = df_country[df_country['year'] == 2023].copy()
    if df_2023.empty:
        print(f"Skipping {country}: No 2023 data available.")
        continue

    # Keep only necessary columns for prediction
    future = df_2023[['ds', 'average', 'max', 'median', 'min']]

    # Predict 2023 population
    forecast = model.predict(future)
    predicted_pop = forecast['yhat'].values[0]

    # Save the result
    results.append({
        'Country': country,
        'Predicted_2023_Population': predicted_pop,
        'Actual_2023_Population': df_2023['population'].values[0]
    })

# Convert results to DataFrame and save
results_prophet = pd.DataFrame(results)
results_prophet.to_csv("predicted_population_2023.csv", index=False)

print("Done! Predictions saved to predicted_population_2023.csv")

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/_lsn68qi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/5isa35jy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=64669', 'data', 'file=/tmp/tmpojhf_0lo/_lsn68qi.json', 'init=/tmp/tmpojhf_0lo/5isa35jy.json', 'output', 'file=/tmp/tmpojhf_0lo/prophet_models0s4nmq2/prophet_model-20250511134955.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
13:49:55 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:49:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:c

Skipping Antigua & Barbuda due to model error: Dataframe has less than 2 non-NaN rows.


13:50:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/oc1aq_x8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/dyu7gtyc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48935', 'data', 'file=/tmp/tmpojhf_0lo/oc1aq_x8.json', 'init=/tmp/tmpojhf_0lo/dyu7gtyc.json', 'output', 'file=/tmp/tmpojhf_0lo/prophet_model1hl_e0x2/prophet_model-20250511135017.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
13:50:17 - cmdstanpy - INFO - Chain [1] start processing
INFO:cm

Skipping Bahamas due to model error: Dataframe has less than 2 non-NaN rows.


13:50:28 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/aad8mfjc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/41htqo74.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=71030', 'data', 'file=/tmp/tmpojhf_0lo/aad8mfjc.json', 'init=/tmp/tmpojhf_0lo/41htqo74.json', 'output', 'file=/tmp/tmpojhf_0lo/prophet_model15ss5d03/prophet_model-20250511135028.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
13:50:28 - cmdstanpy - INFO - Chain [1] start processing
INFO:cm

...

Skipping Venezuela due to model error: Dataframe has less than 2 non-NaN rows.


13:56:54 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/ipi88f5p.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/xk8ujftx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=76081', 'data', 'file=/tmp/tmpojhf_0lo/ipi88f5p.json', 'init=/tmp/tmpojhf_0lo/xk8ujftx.json', 'output', 'file=/tmp/tmpojhf_0lo/prophet_modeluagt9n_d/prophet_model-20250511135654.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
13:56:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cm

Skipping Yemen due to model error: Dataframe has less than 2 non-NaN rows.


13:56:55 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/5ofxqnao.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpojhf_0lo/rghlu2ml.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=95371', 'data', 'file=/tmp/tmpojhf_0lo/5ofxqnao.json', 'init=/tmp/tmpojhf_0lo/rghlu2ml.json', 'output', 'file=/tmp/tmpojhf_0lo/prophet_modelu_d2p8mh/prophet_model-20250511135655.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
13:56:55 - cmdstanpy - INFO - Chain [1] start processing
INFO:cm

Done! Predictions saved to predicted_population_2023.csv


In [42]:
results_prophet= pd.read_csv('/content/predicted_population_2023.csv')
results_prophet.head()

,Country,Predicted_2023_Population,Actual_2023_Population
0,Afghanistan,4.494906e+07,41454761.0
1,Albania,2.748260e+06,2745972.0
2,Algeria,4.398179e+07,46164219.0
3,Andorra,7.902943e+04,80856.0
4,Angola,3.619312e+07,36749906.0


In [44]:
from sklearn.metrics import r2_score,mean_absolute_error

y_true = results_prophet['Actual_2023_Population']
y_pred = results_prophet['Predicted_2023_Population']

# Compute R² score
r2 = r2_score(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
print(f"R² score: {r2:.4f}")
print("MAE:", mae)

R² score: 0.9997
MAE: 641385.7831006938


##Visualization of Results (Prophet Model)

In [61]:
df = results_prophet.copy()

In [65]:
import pandas as pd
import plotly.express as px

# Ground Truth Map
if 'Actual_2023_Population' in df.columns:
    truth_map_df = df[['Country', 'Actual_2023_Population']].copy()
    fig_truth = px.choropleth(truth_map_df, locations='Country',
                              locationmode='country names',
                              color='Actual_2023_Population',
                              color_continuous_scale='Blues',
                              title='Ground Truth Population - 2023')
    fig_truth.show()
else:
    print("Column 'Actual_Truth_2023' not found.")

# Forecast Map
if 'Predicted_2023_Population' in df.columns:
    forecast_map_df = df[['Country', 'Predicted_2023_Population']].copy()
    fig_forecast = px.choropleth(forecast_map_df, locations='Country',
                                 locationmode='country names',
                                 color='Predicted_2023_Population',
                                 color_continuous_scale='Greens',
                                 title='Forecasted Population - 2023')
    fig_forecast.show()
else:
    print("Column 'Forecast_2023' not found.")
